In [ ]:
using Knet: Knet, dir, accuracy, progress, sgd, load, save, gc, Param, KnetArray, Data, minibatch, nll, relu, training, dropout
using CUDA
using LinearAlgebra
using NBInclude
using CUDA

In [ ]:
@nbinclude("deal_features.ipynb")
@nbinclude("model.ipynb")
@nbinclude("explore.ipynb")

In [ ]:
if CUDA.functional()
    atype = KnetArray{Float32};
else
    atype = Array{Float32};
end

In [ ]:
function opening_analysis(model,test_set)
    bid_dict=Dict()
    for i in 1:length(test_set)
        north=test_set[i][1]
        action,_=explore(model[1],atype(north),1,0)
        bid=bids[action]
        try
            push!(bid_dict[bid],north)
        catch
            bid_dict[bid]=[north]
        end
    end 
    return bid_dict
end

In [ ]:

deal_data=load("deals.jld2","deal_data_all");


In [ ]:
model=load("model_highest_feature.jld2","q_networks");

In [ ]:
d=opening_analysis(model,deal_data);

In [ ]:
for bid in keys(d)
    hcps=[]
    n=length(d[bid]);
    dists=zeros(n,4);
    balanced_count=0

    for i in 1:n
        hand=d[bid][i]
        push!(hcps,hcp_of(hand))
        dists[i,:]=distribution_of(hand)
        balanced_count = balanced_count + is_balanced(hand)
    end
    sort!(hcps)
    l=length(hcps)
    s=Int(floor(0.1*l))+1
    e=Int(floor(0.9*l))
    #hcps=hcps[s:e]
    if length(hcps)==0
        continue
    end
    println(bid,": ", minimum(hcps), " - ",maximum(hcps) ,", ", sum(dists,dims=1)./n, ", ", balanced_count/n)
end